# Bird migration analysis example

<img align="right" src="https://anitagraser.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=2-analysis-examples/1-bird-migration.ipynb)

This tutorial uses data published on Movebank, specifically: [Navigation experiments in lesser black-backed gulls (data from Wikelski et al. 2015)-gps.csv](https://www.datarepository.movebank.org/handle/10255/move.494)

This tutorial covers: 
1. Trajectory data preprocessing
 1. Loading movement data from common geospatial file formats
 1. Exploring spatial & non-spatial data distributions
 1. Converting GeoDataFrames into Trajectories describing continuous tracks of moving objects
1. Trajectory data analysis
 1. Investigating individual trajectories
 1. Comparing different years
 1. Investigating trajectories of multiple individuals

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [ ]:
mpd.__version__

## Loading the bird movement data 


In [ ]:
%%time
df = read_file('../data/gulls.gpkg')
print(f"Finished reading {len(df)}")

This is what the data looks like:

In [ ]:
df.head()

In [ ]:
df.plot()

Let's see how many individuals we have in the dataset:

In [ ]:
df['individual-local-identifier'].unique()

The records per individual are not evenly distributed:

In [ ]:
df['individual-local-identifier'].value_counts().plot(kind='bar', figsize=(17,3))

Finally, let's create trajectories:

In [ ]:
traj_collection = mpd.TrajectoryCollection(df, 'individual-local-identifier', t='timestamp', min_length=100)    
traj_collection

And let's generalize them to speed up the following analyses:

In [ ]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(tolerance=timedelta(days=1))

## Investigating individual trajectories

Let's pick out a specific individual. For example, '91916A' is the individual with most records in our dataset:

In [ ]:
filtered = traj_collection.filter('individual-local-identifier', '91916A')
my_traj = filtered.trajectories[0].copy()
my_traj.df.head()

In [ ]:
my_traj.hvplot(title=f'Movement of {my_traj.id}', line_width=2, frame_width=700, frame_height=500) 

This individual has been travelling back and forth for quite a few years!

One way to take a closer look at this individual's travels is to split the overall track into yearly trips:

In [ ]:
trips_by_year = mpd.TemporalSplitter(filtered).split(mode='year')
trips_by_year.to_traj_gdf()

Now we can explore individual years:

In [ ]:
one_year = trips_by_year.get_trajectory('91916A_2010-12-31 00:00:00')
one_year

In [ ]:
one_year.hvplot(title=f'Movement speed of {one_year.id}', frame_width=700, frame_height=500, 
                line_width=5.0, c='speed', cmap='RdYlGn', colorbar=True, clim=(0,10)) 

Let's see where this individual was on a specific day:

In [ ]:
def plot_location_at_timestamp(traj, t, fig_size=250):
    loc = GeoDataFrame([traj.get_row_at(t)])
    return (loc.hvplot(title=str(t), geo=True, tiles='OSM', size=200, color='red', width=fig_size, height=fig_size) * 
            traj.hvplot(line_width=1.0, color='black', tiles=False, width=fig_size, height=fig_size))

In [ ]:
( plot_location_at_timestamp(one_year, datetime(2010,9,1)) + 
  plot_location_at_timestamp(one_year, datetime(2010,10,1)) +
  plot_location_at_timestamp(one_year, datetime(2010,11,1)) )

Of course, it might also be of interest to see the different locations on a certain day each year:

In [ ]:
def plot_location_at_day_of_year(traj, month, day, ax=None):
    ts = [datetime(year, month, day) for year in traj.df.index.year.unique()]
    return plot_locations_at_timestamps(traj, ts, ax=ax)

def plot_locations_at_timestamps(traj, ts, ax=None):  
    loc = GeoDataFrame([traj.get_row_at(t) for t in ts])
    loc['date_label'] = loc.index.strftime('%Y-%m-%d')
    return (loc.hvplot(title=f'Movement of {traj.id}', c='date_label', size=200, geo=True, tiles='OSM') *
            traj.hvplot(line_width=1.0, color='black', geo=True, tiles=False, frame_width=700, frame_height=500) )

In [ ]:
plot_location_at_day_of_year(my_traj, month=10, day=1)

It's pretty clear that this individual does not follow the same schedule and route every year. However, it seems to always be heading to the same area Red Sea coast to spend the winter there.

Let's find its arrival times in this area:

In [ ]:
area_of_interest = Polygon([(30, 25), (50, 25), (50, 15), (30, 15), (30, 25)])
plotted_area_of_interest = GeoDataFrame(pd.DataFrame([{'geometry': area_of_interest, 'id': 1}]), crs=4326).hvplot(geo=True, color='yellow', alpha=0.5)

In [ ]:
arrivals = [traj for traj in my_traj.clip(area_of_interest)]
print(f"Found {len(arrivals)} arrivals")

for traj in arrivals:
    print(f"Individual '{traj.df['individual-local-identifier'].iloc[0]}' arrived at {traj.get_start_time()}")

In [ ]:
( plot_locations_at_timestamps(my_traj, [traj.get_start_time() for traj in arrivals]) * plotted_area_of_interest )

## Investigating trajectories of multiple individuals

Multiple individuals travel to this area every year. Let's have a closer look:

In [ ]:
year_of_interest = 2010
trajs_in_aoi = traj_collection.clip(area_of_interest)
relevant = [ traj for traj in trajs_in_aoi if traj.get_start_time().year <= year_of_interest and traj.get_end_time().year >= year_of_interest]
print("Found {} arrivals".format(len(relevant)))

In [ ]:
for traj in relevant:
    print("Individual '{}' arrived at {} (duration: {})".format(
        traj.df['individual-local-identifier'].iloc[0], traj.get_start_time().date(), 
        traj.get_end_time()-traj.get_start_time()))

Based on the duration of the individuals' trajectory segments within our area of interest, it looks like some individuals spend the winter here while others only pass through.

For example, Individual '91761A' passed through twice? What has it been up to?

In [ ]:
my_traj = traj_collection.get_trajectory('91761A')
segment = my_traj.get_segment_between(datetime(year_of_interest,1,1), datetime(year_of_interest,12,31))

segment.hvplot(color='black', line_width=1.0, frame_width=700, frame_height=500) * plotted_area_of_interest 

Turns out that this individual does not stay at the Red Sea but continues its journey into Africa.

## Continue exploring MovingPandas

1. [Bird migration analysis](1-bird-migration.ipynb)
1. [Ship data analysis](2-ship-data.ipynb)
1. [Horse collar data exploration](3-horse-collar.ipynb)
1. [Stop hotspot detection](4-stop-hotspots.ipynb)
1. [OSM traces](5-osm-traces.ipynb)